In [1]:
import os
import ast
import zipfile
import textwrap
import subprocess
import configparser
import pandas as pd
from queries import *

from neo4j import GraphDatabase

# Data Preparation and Neo4j Ingest

In this notebook we format the 
[Food.com Recipes and Interactions Dataset](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) [[1]](#1) and load it into a Neo4j graph. The work is divided into two primary sections in this notebook:

1. __[Prepare Data (Extract & Transform)](#prepare-data)__
2. __[Load Data Into Neo4j](#load-data)__

__To run this notebook please do the following:__
 - Download the training and validation sets from [Food.com Recipes and Interactions Dataset](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) and place into the `SOURCE_DATA_PATH` set in the below [Setup Parameters](#set-up) section (optional if kaggle API keys are in the system).

 - Ensure the `NEO4J_IMPORT_DIR`, `NEO4j_IMPORT_PATH`, and `NEO4J_PROPERTIES_FILE` are set correctly for your environemnt in the below [Setup Parameters](#set-up) section.

__Other Prerequisites:__
- Neo4j. This notebook was tested with [Neo4j Desktop](https://neo4j.com/download-center/#desktop) and should work for other on-prem installations.
- [APOC library](https://neo4j.com/labs/apoc/4.3/installation/). 
- [GDS Library](https://neo4j.com/docs/graph-data-science/current/installation/neo4j-desktop/).
- Notebook tested with Python=3.10

<a id="1">[1]</a>
Bodhisattwa Prasad Majumder, Shuyang Li, Jianmo Ni, and Julian McAuley. 2019. Generating Personalized Recipes from Historical User Preferences. In Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP), pages 5976–5982, Hong Kong, China. Association for Computational Linguistics.

## <a id="set-up">Setup Parameters</a>

#### Directory Settings

In [2]:
# Data directory
SOURCE_DATA_PATH = './data/'

`NEO4J_IMPORT_DIR` sets the the directory to put formatted csvs in for Neo4j load.
File locations for each OS are listed in the [Neo4j Operations Manual](https://neo4j.com/docs/operations-manual/current/configuration/file-locations). We recommend putting files in Neo4j’s `import` directory, as it keeps the environment secure. However, if you need to access files in other locations, you can find out which setting to alter in our [manual](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/#query-load-csv-introduction).

In [3]:
# Replace with your Neo4j import directory
NEO4J_IMPORT_DIR = '/home/luigi/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-c5c50094-d809-4eeb-84f4-745becc1cd52/import'

`NEO4j_IMPORT_PATH` sets the path to use during CSV load. This will be 'file:///' by default and aligns with the local Neo4j `import` directory setup. 
See https://neo4j.com/developer/guide-import-csv/#_reading_csv_files for more details and customization

In [4]:
NEO4j_IMPORT_PATH = 'file:///'

#### Neo4j Settings
The `NEO4J_PROPERTIES_FILE` is an ini configuration file for Neo4j properties so this notebook can connect to
your Neo4j instance and load data. The ini file should be formatted as follows
```
[NEO4J]
PASSWORD=<password>
DATABASE=<database name, is 'neo4j' by default>
HOST=<host uri>

```

Set `NEO4J_PROPERTIES_FILE` to None or any non-existent location to use the below defaults
```
HOST = 'neo4j://localhost'
DATABASE = 'neo4j'
PASSWORD = 'password'
```

In [5]:
NEO4J_PROPERTIES_FILE = None # 'path/to/your/neo4j_properties.ini'

#### Impression Chunk Size
The number of rows to work with at once when reading from source behaviors/impressions files. Recommend keeping
this below 100,000 - ish unless you know your machine can handle more rows efficiently.

In [6]:
IMPRESSION_CHUNK_SIZE = 100_000

## <a id="prepare-data">Prepare Data (Extract & Transform)</a>
In this section we extract and transform the source data into formatted csv files to prepare for [loading into Neo4j](#load-data).
We will create the following csv files:

__Recipes related graph elements__
1. `recipe.csv`: Nodes representing recipes
2. `ingredients.csv`: Nodes representing ingredients
3. `with_ingredients.csv`: Relationship representing recipe having a set of ingredients
4. `submitted.csv`: Relationship representing user creating (posting) a recipe
 
__User and impression related graph elements__
1. `users.csv`: Nodes representing anonymized users
2. `reviewed.csv`: Relationship representing user reviewing a recipe

### Download or extract needed data files

In [7]:
try:
    # If the zip dataset has not been downloaded, download it
    if not os.path.exists(os.path.join(SOURCE_DATA_PATH, 'archive.zip')):
        # Create data directory
        os.makedirs(SOURCE_DATA_PATH, exist_ok=True)
        # Download the dataset
        subprocess.run(f'kaggle datasets download -d shuyangli94/food-com-recipes-and-user-interactions --path {SOURCE_DATA_PATH}', shell=True, check=True)
        # Rename dataset to archive.zip
        os.rename(os.path.join(SOURCE_DATA_PATH, "food-com-recipes-and-user-interactions.zip"), os.path.join(SOURCE_DATA_PATH, "archive.zip"))
        # Print message
        print("Dataset downloaded successfully.")

    # Extract zip file to SOURCE_DATA_PATH
    with zipfile.ZipFile(os.path.join(SOURCE_DATA_PATH, "archive.zip"), "r") as zip_ref:
        zip_ref.extractall(SOURCE_DATA_PATH)
    # Print message
    print("Dataset extracted successfully.")

except FileNotFoundError as fnf_error:
    print(f"Error: {fnf_error}")
    print("Kaggle API credentials file not found. Please ensure kaggle.json is configured correctly.")
except subprocess.CalledProcessError as cpe_error:
    print(f"Error during dataset download: {cpe_error}")
    print("There was an error downloading the dataset. Please check your Kaggle API setup and dataset path.")
except Exception as e:
    print(f"An error occurred: {e}")

Dataset extracted successfully.


### Recipe Related Graph Elements

Let's start by uploading the raw recipes csv file as a dataframe

In [8]:
# Read RAW_recipes.csv file to raw_recipes_df dataframe
raw_recipes_df = pd.read_csv(os.path.join(SOURCE_DATA_PATH, 'RAW_recipes.csv'))
raw_recipes_df.head(n=2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


#### Recipe Nodes
For the recipe node we will basically just drop some columns from the raw_recipes_df and just keep the following columns:
1. `name`: recipe name
2. `id`: recipe id
3. `minutes`: minutes needed to prepare the recipe
4. `tags`: food.com tags for the recipe
5. `nutrition`: nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat)
6. `n_steps`: number of steps in the recipe
7. `steps`: text for recipe steps in order
8. `description`: user-provided description of the recipe
9. `n_ingredients`: number of ingredients needed

In [9]:
# Create recipes dataframe as a copy to avoid SettingWithCopyWarning
recipes_df = raw_recipes_df[['name', 'id', 'minutes', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients']].copy()

# Set default values without using inplace=True
recipes_df['name'] = recipes_df['name'].fillna('No Name')
recipes_df['description'] = recipes_df['description'].fillna('No Description')

# Save as csv file to NEO4J_IMPORT_DIR
recipes_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'recipes.csv'), index=False)

# Show dataframe
recipes_df.head(n=2)

,name,id,minutes,tags,nutrition,n_steps,steps,description,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,7
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,6


#### Ingredients Nodes
For the ingredients node we will create a set of ingredients iterating all recipies and create a dataframe with the following colums:
1. `id`: ingredient ID
2. `name`: ingredient name

In [10]:
# Create an empty set to store unique ingredients
ingredients_dict = dict()
id_counter = 1

# Iterate over all lists of ingredients
for ingredients_list in raw_recipes_df['ingredients']:
    # Iterate over all ingredients
    for ingredient in ast.literal_eval(ingredients_list):
        # Add ingredient to the dictionary (if not present)
        if ingredient not in ingredients_dict:
            ingredients_dict[ingredient] = id_counter
            id_counter += 1

# Create a dataframe from the ingredients dictionary
ingredients_df = pd.DataFrame(list(ingredients_dict.items()), columns=['name', 'id'])
# Save as csv file to NEO4J_IMPORT_DIR
ingredients_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'ingredients.csv'), index = False)
# Show dataframe
ingredients_df.head(n=2)

,name,id
0,winter squash,1
1,mexican seasoning,2


#### WithIngredients Relationship
For the with_ingredients relationship we will instead create a dataframe with the following columns:
1. `recipe_id`: id of the recipe
2. `ingredient_id`: name of the ingredient

In [11]:
# Initialize an empty list for 'WITH_INGREDIENTS' relation
recipe_ingr_data = []

# Iterate through raw_recipes_df recipes
for index, row in raw_recipes_df.iterrows():
    # Iterate through the recipe's ingredients list
    for ingredient in ast.literal_eval(row['ingredients']):
        # Add to relation the entry <recipe_id>, <ingredient_id>
        recipe_ingr_data.append({'recipe_id':row['id'], 'ingredient_id':ingredients_dict[ingredient]})

# Create with_ingredients dataframe from recipe_ingr_data
with_ingredients_df = pd.DataFrame(recipe_ingr_data)
# Save as csv file to NEO4J_IMPORT_DIR
with_ingredients_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'with_ingredients.csv'), index = False)
# Show dataframe
with_ingredients_df.head(n=2)

,recipe_id,ingredient_id
0,137739,1
1,137739,2


#### Submitted Relationships

The submitted relationship will include data about which user submitted a recipe with the following columns:
1. `recipe_id`: recipe ID
2. `user_id`: user ID of the contributor (user who submitted the recipe)
3. `submitted`: date recipe was submitted

In [12]:
# Create submitted_df dataframe
submitted_df = raw_recipes_df[['id', 'contributor_id', 'submitted']].drop_duplicates()
# Rename columns 
submitted_df = submitted_df.rename(columns={'id': 'recipe_id', 'contributor_id':'user_id'})
# Save as csv file to NEO4J_IMPORT_DIR
submitted_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'submitted.csv'), index = False)
# Show dataframe
submitted_df.head(n=2)

,recipe_id,user_id,submitted
0,137739,47892,2005-09-16
1,31490,26278,2002-06-17


### User and Impression Related Graph Elements

The reviewed relation is just given by the RAW_interactions.csv, hence we can keep it as it is with the following columns:
1. `user_id`: user_id
2. `recipe_id`: recipe_id
3. `date`: date of interaction
4. `rating`: given rating (0 to 5)
5. `review`: review text

In [13]:
# Read RAW_interactions.csv file into a DataFrame
full_reviewed_df = pd.read_csv(os.path.join(SOURCE_DATA_PATH, 'RAW_interactions.csv'))

# Convert the date column to datetime format if it's not already
full_reviewed_df['date'] = pd.to_datetime(full_reviewed_df['date'])

# Set a minimum number of interactions threshold
min_interactions = 5
# Filter users by minimum interaction count
user_counts = full_reviewed_df['user_id'].value_counts()
eligible_users = user_counts[user_counts >= min_interactions].index
filtered_df = full_reviewed_df[full_reviewed_df['user_id'].isin(eligible_users)]

# Determine the split date for, say, the last 20% of interactions
split_date = filtered_df['date'].quantile(0.8)
reviewed_df = filtered_df[filtered_df['date'] <= split_date]
test_reviewed_df = filtered_df[filtered_df['date'] > split_date]

# Ensure test users are also in train
test_users_in_train = test_reviewed_df['user_id'].isin(reviewed_df['user_id'])
test_reviewed_df = test_reviewed_df[test_users_in_train]

# Create a copy to avoid SettingWithCopyWarning
reviewed_df = reviewed_df.copy()
# Set default values
reviewed_df['review'] = reviewed_df['review'].fillna('No Review')
# Remove backslashes from the 'review' column
reviewed_df['review'] = reviewed_df['review'].str.replace('\\', '')

# Save train_df and test_df as CSV files
reviewed_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'reviewed.csv'), index=False)
test_reviewed_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'test_reviewed.csv'), index=False)

# Show dataframes
print(reviewed_df.head(n=2))
print(test_reviewed_df.head(n=2))

   user_id  recipe_id       date  rating  \
0    38094      40893 2003-02-17       4   
2     8937      44394 2002-12-01       4   

                                              review  
0  Great with a salad. Cooked on top of stove for...  
2  This worked very well and is EASY.  I used not...  
   user_id  recipe_id       date  rating  \
4    57222      85009 2011-10-01       5   
6   124416     120345 2011-08-06       0   

                                              review  
4  Made the cheddar bacon topping, adding a sprin...  
6  Just an observation, so I will not rate.  I fo...  


In [14]:
'''
# Read RAW_recipes.csv file to raw_recipes_df dataframe
full_reviewed_df = pd.read_csv(os.path.join(SOURCE_DATA_PATH, 'RAW_interactions.csv'))

# Split the dataframe into train and test sets
reviewed_df = full_reviewed_df.sample(frac=0.8, random_state=42)
test_reviewed_df = full_reviewed_df.drop(reviewed_df.index)

# Save test_reviewed_df as csv file to NEO4J_IMPORT_DIR
test_reviewed_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'test_reviewed.csv'), index = False)

# Print the length of the train and test sets to check if the split was successful
print(f"Initial set length: {len(full_reviewed_df)}")
print(f"Train set length: {len(reviewed_df)}")
print(f"Test set length: {len(test_reviewed_df)}")

# Set default values
reviewed_df['review'] = reviewed_df['review'].fillna('No Review')
# Remove backslashes from the 'review' column
reviewed_df['review'] = reviewed_df['review'].str.replace('\\', '')
# Save as csv file to NEO4J_IMPORT_DIR
reviewed_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'reviewed.csv'), index = False)
# Show dataframe
reviewed_df.head(n=2)
'''

'\n# Read RAW_recipes.csv file to raw_recipes_df dataframe\nfull_reviewed_df = pd.read_csv(os.path.join(SOURCE_DATA_PATH, \'RAW_interactions.csv\'))\n\n# Split the dataframe into train and test sets\nreviewed_df = full_reviewed_df.sample(frac=0.8, random_state=42)\ntest_reviewed_df = full_reviewed_df.drop(reviewed_df.index)\n\n# Save test_reviewed_df as csv file to NEO4J_IMPORT_DIR\ntest_reviewed_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, \'test_reviewed.csv\'), index = False)\n\n# Print the length of the train and test sets to check if the split was successful\nprint(f"Initial set length: {len(full_reviewed_df)}")\nprint(f"Train set length: {len(reviewed_df)}")\nprint(f"Test set length: {len(test_reviewed_df)}")\n\n# Set default values\nreviewed_df[\'review\'] = reviewed_df[\'review\'].fillna(\'No Review\')\n# Remove backslashes from the \'review\' column\nreviewed_df[\'review\'] = reviewed_df[\'review\'].str.replace(\'\\\', \'\')\n# Save as csv file to NEO4J_IMPORT_DIR\nreviewed_df.to_

#### User Nodes

For the user nodes, we will just keep unique 'user_id' from the ratings data:

In [15]:
# Create user dataframe
user_df = reviewed_df[['user_id']].drop_duplicates().rename(columns={'user_id': 'id'})
# Save as csv file to NEO4J_IMPORT_DIR
user_df.to_csv(os.path.join(NEO4J_IMPORT_DIR, 'users.csv'), index = False)
# Show dataframe
user_df.head(n=2)

,id
0,38094
2,8937


## <a id="load-data">Load Data Into Neo4j</a>

In this section we will load the above transformed data into Neo4j. As a good rule of practice for Neo4j bulk loading,
I have found it ideal if you can follow the below pattern. Note that for this to work, each node should have some sort
of external unique id, or you must otherwise create one at ingest time.

1. __Set a uniqueness constraint__ on a property for each node label
2. __Load nodes__ using `MERGE` which is similar to an upsert pattern on the above uniqueness constraint and is very well optimized in Neo4j
3. __Load relationships__ using `MATCH` and `MERGE` to find adjacent nodes on uniqueness constraint and upsert respectively

In [16]:
# Using an ini file for credentials, otherwise providing defaults
HOST = 'bolt://localhost:7687'
DATABASE = 'neo4j'
PASSWORD = 'LU?vVtYf8])9;^z'

if NEO4J_PROPERTIES_FILE is not None and os.path.exists(NEO4J_PROPERTIES_FILE):
    config = configparser.RawConfigParser()
    config.read(NEO4J_PROPERTIES_FILE)
    HOST = config['NEO4J']['HOST']
    DATABASE = config['NEO4J']['DATABASE']
    PASSWORD = config['NEO4J']['PASSWORD']
    print('Using custom database properties')
else:
    print('Could not find database properties file, using defaults')

Could not find database properties file, using defaults


In [17]:
# helper function
def run(driver, query, params=None):
    with driver.session() as session:
        if params is not None:
            return [r for r in session.run(query, params)]
        else:
            return [r for r in session.run(query)]

In [18]:
driver = GraphDatabase.driver(HOST, auth=(DATABASE, PASSWORD))

### Set Uniqueness Constraints
First, let's clear the database

In [19]:
# Delete all previous nodes and relations
run(driver, textwrap.dedent("""\
    CALL apoc.periodic.commit(
        "MATCH (n)
        WITH n LIMIT $limit
        DETACH DELETE n 
        RETURN COUNT(*)", 
    {limit:1000})
    """))

[<Record updates=438778 executions=439 runtime=17 batches=440 failedBatches=0 batchErrors={} failedCommits=0 commitErrors={} wasTerminated=False>]

### Set Uniqueness Constraints
This step is critical for performant loading with merge patterns

In [20]:
run(driver,'CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (user:User) REQUIRE user.id  IS UNIQUE')
run(driver,'CREATE CONSTRAINT recipe_id_unique IF NOT EXISTS FOR (recipe:Recipe) REQUIRE recipe.id IS UNIQUE')
run(driver,'CREATE CONSTRAINT ingredient_id_unique IF NOT EXISTS FOR (ingredient:Ingredient) REQUIRE ingredient.id IS UNIQUE')

[]

### Load Nodes

In [21]:
# User nodes
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    WITH row
    CALL { 
        WITH row  
        MERGE(user:User {id:row.id})
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'users.csv')}
)

[]

In [22]:
# Recipe nodes
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    WITH row
    CALL { 
        WITH row
        MERGE(recipe:Recipe {
            id: row.id,
            name: row.name,
            minutes: row.minutes,
            tags: row.tags,
            nutrition: row.nutrition,  // apoc.convert.fromJsonList(row.nutrition)
            n_steps: row.n_steps,
            steps: row.steps,
            description: row.description,
            n_ingredients: row.n_ingredients })
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'recipes.csv')}
)

[]

In [23]:
# Ingredient nodes
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    WITH row
    CALL { 
        WITH row
        MERGE(ingredient:Ingredient {
            id: row.id,
            name: row.name })
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'ingredients.csv')}
)

[]

### Load Relationships

In [24]:
# WithIngredients relationship
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    WITH row
    CALL { 
        WITH row
        MATCH(recipe:Recipe {id: row.recipe_id})
        MATCH(ingredient:Ingredient {id: row.ingredient_id})
        MERGE(recipe)-[r:WITH_INGREDIENTS]->(ingredient)
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'with_ingredients.csv')}
)

[]

In [25]:
# Submitted relationship
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    WITH row
    CALL { 
        WITH row        
        MATCH(user:User {id: row.user_id})
        MATCH(recipe:Recipe {id: row.recipe_id})
        MERGE(user)-[r:SUBMITTED {submitted: date(row.submitted)}]->(recipe)
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'submitted.csv')}
)

[]

In [26]:
# Rewiewed relationship
run(driver, textwrap.dedent("""\
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file AS row RETURN row',
        'MERGE (user:User {id: row.user_id})
        MERGE (recipe:Recipe {id: row.recipe_id})
        MERGE (user)-[r:REVIEWED {
            date: date(row.date),
            rating: toInteger(row.rating),
            review: row.review
        }]->(recipe)',
        {batchSize:10000, parallel:true, params: {file: $file}}
    )
    """),
    params = {'file': os.path.join(NEO4j_IMPORT_PATH, 'reviewed.csv')}
)

[<Record batches=70 total=697656 timeTaken=17 committedOperations=30000 failedOperations=667656 failedBatches=67 retries=0 errorMessages={"ForsetiClient[transactionId=158097, clientId=69] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=158123, clientId=77]} on NODE_RELATIONSHIP_GROUP_DELETE(416250) because holders of that lock are waiting for ForsetiClient[transactionId=158097, clientId=69].\n Wait list:ExclusiveLock[\nClient[158123] waits for [ForsetiClient[transactionId=158097, clientId=69],ForsetiClient[transactionId=158120, clientId=74],ForsetiClient[transactionId=158106, clientId=61]]]": 1, "ForsetiClient[transactionId=158114, clientId=21] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=158106, clientId=61]} on NODE_RELATIONSHIP_GROUP_DELETE(416112) because holders of that lock are waiting for ForsetiClient[transactionId=158114, clientId=21].\n Wait list:ExclusiveLock[\nClient[158106] waits for [ForsetiClient[transactionId=158105, clientId=66],ForsetiCl

In [27]:
driver.close()